In [57]:
import pandas as pd 
import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline


# Предобработка и соединение файлов с номерами и датами

In [58]:
quart = pd.DataFrame()

for file in os.listdir():
    if '$' not in file and 'xlsx' in file and 'АУ.xlsx' not in file:
        dict_df = pd.read_excel(file, sheet_name=None, dtype=str)
        for key in dict_df.keys():
            name = (file[: file.find('.')] + ' ' + key).strip()
            temp_df = dict_df[key][['Дата', 'Клиент: телефон']]
            temp_df['Источник'] = [name] * temp_df.shape[0]
            quart = quart.append(temp_df)

C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [59]:
quart = quart.reset_index(drop=True)


In [60]:
pattern = re.compile(r'([^ ]+)( [^ ]+)')


def string(s):
    pattern = re.compile(r'([^ ]+)( [^ ]+)')
    if pattern.findall(s):
        return pattern.findall(s)[0][0]
    else:
        return s
    
quart['Дата_new'] = quart['Дата'].apply(string)


In [61]:
def to_date(x):
    try:
        x = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    except:
        x = datetime.datetime.strptime(x, '%d.%m.%Y').date()
    return x    

In [62]:
quart['Дата_new'] = quart['Дата_new'].apply(to_date)

In [63]:
quart.drop('Дата', axis=1, inplace=True)

In [64]:
quart = quart.rename(columns={'Дата_new': 'Дата'})

In [65]:
quart

,Клиент: телефон,Источник,Дата
0,79612200737,Звонки 4кв 2гис НСК Звонки,2020-12-31
1,79132056199,Звонки 4кв 2гис НСК Звонки,2020-12-31
2,79525029610,Звонки 4кв 2гис НСК Звонки,2020-12-31
3,79137623129,Звонки 4кв 2гис НСК Звонки,2020-12-31
4,79529420311,Звонки 4кв 2гис НСК Звонки,2020-12-31
...,...,...,...
12923,89130031521,Звонки 4кв Яндекс НСК Заявки,2020-10-08
12924,89529469395,Звонки 4кв Яндекс НСК Заявки,2020-10-12
12925,89137953532,Звонки 4кв Яндекс НСК Заявки,2020-10-20
12926,89994503226,Звонки 4кв Яндекс НСК Заявки,2020-10-26


In [66]:
quart = quart.dropna().reset_index(drop=True)


# re - телефоны

In [68]:
phones = quart['Клиент: телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y    

In [69]:
quart['длина'] = quart['Клиент: телефон'].apply(len)

In [70]:
quart['Чистые телефоны'] = phones2

In [71]:
quart['Чистые телефоны'] = quart['Чистые телефоны'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

In [72]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [73]:
quart[quart['длина чистые'] != 11]






,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые
9531,нет,Звонки 4кв 2гис НСК Заявки,2020-10-02,3,,0
9542,нет,Звонки 4кв 2гис НСК Заявки,2020-10-22,3,,0
9544,нет,Звонки 4кв 2гис НСК Заявки,2020-10-25,3,,0
9545,нет,Звонки 4кв 2гис НСК Заявки,2020-10-25,3,,0
9548,нет,Звонки 4кв 2гис НСК Заявки,2020-10-28,3,,0
9551,нет,Звонки 4кв 2гис НСК Заявки,2020-11-04,3,,0
12688,441143499024,Звонки 4кв Яндекс НСК Звонки,2020-10-27,12,441143499024,12


In [75]:
quart = quart.drop(quart[quart['длина чистые'] != 11].index)

In [78]:
def clean_number(row):
    if row['длина чистые'] == 14:
        row['Чистые телефоны'] = '7' + row['Чистые телефоны'][4:]
    return row
        

In [79]:
quart = quart.apply(clean_number, axis=1)

In [80]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [81]:
pattern_7 = r'^7\d{10}$'
quart['соответствие'] = quart['Чистые телефоны'].apply(lambda x: bool(re.match(pattern_7, x)))

In [82]:
quart[~quart['соответствие']]

,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые,соответствие
40,15416455163,Звонки 4кв 2гис НСК Звонки,2020-12-30,11,15416455163,11,False


In [83]:
quart.drop([40], inplace=True)


In [84]:
quart.drop(['Клиент: телефон', 'длина', 'длина чистые', 'соответствие'], axis=1, inplace=True)

In [85]:
quart = quart.sort_values(by='Дата', ignore_index=True)
quart = quart.drop_duplicates(subset='Чистые телефоны', ignore_index=True)
quart

,Источник,Дата,Чистые телефоны
0,Звонки 4кв 2гис НСК Звонки,2020-10-01,79139135417
1,Звонки 4кв 2гис НСК Звонки,2020-10-01,73832049013
2,Звонки 4кв 2гис НСК Звонки,2020-10-01,79231515015
3,Звонки 4кв 2гис НСК Звонки,2020-10-01,79137958932
4,Звонки 4кв 2гис НСК Звонки,2020-10-01,79529299457
...,...,...,...
6046,Звонки 4кв 2гис НСК Звонки,2020-12-31,79525029610
6047,Звонки 4кв 2гис НСК Звонки,2020-12-31,79529420311
6048,Звонки 4кв 2гис НСК Звонки,2020-12-31,79137533519
6049,Звонки 4кв Гугл НСК Звонки,2020-12-31,79293817976


# АУ

In [86]:
au_telefons = pd.read_excel('АУ.xlsx')
au_telefons = au_telefons.fillna('')

In [87]:
dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

In [88]:
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if len([x for x in j if x.isdigit()]) == 10:
            key[i] = '7' + ''.join([x for x in j if x.isdigit()])    
        if len([x for x in j if x.isdigit()]) == 11:
            temp_arr = ''.join([x for x in j if x.isdigit()])
            if temp_arr[0] == '8':
                key[i] = '7' + temp_arr[1:]
            elif temp_arr[0] == '7':    
                key[i] = temp_arr




for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  

In [89]:
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j

In [90]:
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)        

In [91]:
lst_search

['79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 '79139411158 79133701469 ',
 ' 79139133489 ',
 '73832066471 79232568200 ',
 '73832066471 79232568200 ',
 '73832066471 79232568200 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79538603500 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79232403979 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79139877419 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79134558016 ',
 ' 79538753603 ',
 ' 79232403980 ',
 ' 79139396377 ',
 ' 79139396377 ',
 '73832166557 79139262716 

# Сравнение

In [92]:
lst_with_coincidenc = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc.append('')
    for i in quart['Чистые телефоны']:
        if str(i) in row:
            lst_with_coincidenc[index] = str(i)
            break
            
            
    

In [93]:
au_telefons['Совпадение'] = lst_with_coincidenc

In [94]:
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']

In [95]:
au_telefons_final = au_telefons_final.merge(quart, left_on='Совпадение', right_on='Чистые телефоны', how='left')

In [96]:
au_telefons_final = au_telefons_final.fillna('')

In [97]:
au_telefons_final['Старый/Новый'] = au_telefons_final['Дата приема'] == au_telefons_final['Минимальная дата приема Пациента (без удаленных)']
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})
new_clients_lst = au_telefons_final[au_telefons_final['Старый/Новый'] == 'Новый']['Номер карты'].unique()
au_telefons_final.drop('Старый/Новый', axis=1, inplace=True)
au_telefons_final['Старый/Новый'] = au_telefons_final['Номер карты'].isin(new_clients_lst)
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})


In [98]:
au_telefons_final

,Номер карты,Пациент,Дата приема,Минимальная дата приема Пациента (без удаленных),Наименование услуги,Отделение из профиля Исполнителя,Телефон домашний,Телефон мобильный,Телефон рабочий,Категория клиента,Sum([Стоимость полная]),Sum([Количество]),Филиал,Совпадение,Источник,Дата,Чистые телефоны,Старый/Новый
0,540000945,Антошкина Анна Александровна,2020-11-21,2011-02-02,"Консультация врача - гинеколога, зав.отд., вед...",Гинекологическое отделение,,+7(923)240-39-79,,СК,1600.0,1,Некрасова,79232403979,Звонки 4кв 2гис НСК Звонки,2020-10-01,79232403979,Старый
1,540000945,Антошкина Анна Александровна,2020-11-21,2011-02-02,- Мазок цитологический,Гинекологическое отделение,,+7(923)240-39-79,,СК,505.0,1,Некрасова,79232403979,Звонки 4кв 2гис НСК Звонки,2020-10-01,79232403979,Старый
2,540000945,Антошкина Анна Александровна,2020-11-21,2011-02-02,Взятие гинекологических мазков в рамках консул...,Гинекологическое отделение,,+7(923)240-39-79,,СК,500.0,1,Некрасова,79232403979,Звонки 4кв 2гис НСК Звонки,2020-10-01,79232403979,Старый
3,540000945,Антошкина Анна Александровна,2020-11-21,2011-02-02,- Мазок на микрофлору,Гинекологическое отделение,,+7(923)240-39-79,,СК,400.0,1,Некрасова,79232403979,Звонки 4кв 2гис НСК Звонки,2020-10-01,79232403979,Старый
4,540000945,Антошкина Анна Александровна,2020-11-29,2011-02-02,Забор крови из вены в амбулатории,Сестринская служба: Отделение процедурных медс...,,+7(923)240-39-79,,СК,250.0,1,Некрасова,79232403979,Звонки 4кв 2гис НСК Звонки,2020-10-01,79232403979,Старый
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7200,550003790,Стасышина Татьяна Леонидовна,2020-12-14,2020-12-14,Консультация врача - терапевта первичная на дому,Терапевтическое отделение,,+7(913)908-44-96,,НАЛ,2700.0,1,Выборная,79139084496,Звонки 4кв Яндекс НСК Звонки,2020-12-14,79139084496,Новый
7201,550003799,Гуляева Ксения Сергеевна,2020-12-18,2020-12-18,- Общий анализ мочи (ОАМ),Сестринская служба: Отделение процедурных медс...,,+7(951)372-44-00,,НАЛ,230.0,1,Выборная,79513724400,Звонки 4кв 2гис НСК Звонки,2020-10-08,79513724400,Новый
7202,550003799,Гуляева Ксения Сергеевна,2020-12-18,2020-12-18,Прием и оформление биоматериала,Сестринская служба: Отделение процедурных медс...,,+7(951)372-44-00,,НАЛ,20.0,1,Выборная,79513724400,Звонки 4кв 2гис НСК Звонки,2020-10-08,79513724400,Новый
7203,550003824,Шишина Марина Ивановна,2020-12-22,2020-12-22,- Общий анализ мочи (ОАМ),Сестринская служба: Отделение процедурных медс...,,+7(913)789-00-98,,НАЛ,230.0,1,Выборная,79137890098,Звонки 4кв 2гис НСК Звонки,2020-10-10,79137890098,Новый


In [99]:
au_telefons_final = au_telefons_final[au_telefons_final['Дата'] <= au_telefons_final['Дата приема']]

In [100]:
au_telefons_final['Уникальный клиент'] = (~au_telefons_final.duplicated(subset=['Номер карты'], keep='first')).astype(int)


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
au_telefons_final.to_excel('Обработано-НСК.xlsx', index=False)